## Install and import dependencies

In [1]:
#%pip install torch gensim datasets nltk

In [1]:
import os
import nltk
nltk.download("all")


import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import gensim.downloader as api

from datasets import load_dataset
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from torch.utils.data import Dataset, DataLoader


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/mythilimulani/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/mythilimulani/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/mythilimulani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/mythilimulani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/mythilimulani/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagg

## Part 0. Dataset Preparation

In [2]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train'] 
validation_dataset = dataset['validation']
test_dataset = dataset['test']

### Dataset Exploration

In [3]:
#Number of sentences in each set 
print(f"Size of training set: {train_dataset.num_rows} sentences")
print(f"Size of validation set: {validation_dataset.num_rows} sentences")
print(f"Size of test set: {test_dataset.num_rows} sentences")

Size of training set: 8530 sentences
Size of validation set: 1066 sentences
Size of test set: 1066 sentences


In [4]:
print(f"Sample sentence from train dataset: {test_dataset[0]['text']}")
print(f"Label: {'Positive' if test_dataset[0]['label'] == 1 else 'Negative'}")

Sample sentence from train dataset: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
Label: Positive


## Part 1. Preparing Word Embeddings

### Question 1 Word Embedding

#### (a) What is the size of the vocabulary formed in your training data

In [5]:
#tokenize sentences 
train_tokenized = []
for sentence in train_dataset['text']:
    train_tokenized.append(word_tokenize(sentence.lower()))

print('sample sentence:', train_tokenized[0],'\n')

#build vocabulary
vocab = {"<PAD>", "<UNK>"} #include a padding and unknown token for future processing
vocab.update(word for sentence in train_tokenized for word in sentence)

print("Number of words in the vocabulary(including padding and unknown tokens):", len(vocab))
print("Number of words in the vocabulary:" , len(vocab)-2)


sample sentence: ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'s", 'new', '``', 'conan', '``', 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.'] 

Number of words in the vocabulary(including padding and unknown tokens): 18031
Number of words in the vocabulary: 18029


#### (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

#### (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

In [11]:
# Load pretrained Word2Vec model (Google News Word2Vec)
word2vec = api.load('word2vec-google-news-300')

# Set embedding size
embedding_size = 300

# Initialize the embedding matrix with zeros for padding and random values for unknown tokens
embedding_matrix = {}

# Create an <UNK> token embedding as a random vector
unk_vector = np.random.uniform(-0.25, 0.25, embedding_size)
embedding_matrix["<UNK>"] = unk_vector

# Create a <PAD> token embedding as a zero vector
pad_vector = np.zeros(embedding_size)
embedding_matrix["<PAD>"] = pad_vector

# Initialize OOV counter
oov_count = 0

# Iterate over the vocabulary
for word in vocab:
    if word == "<PAD>" or word == "<UNK>":
        continue  
    
    if word in word2vec:  # If the word is in Word2Vec, add its embedding
        embedding_matrix[word] = word2vec[word]
    else:
        # If the word is OOV, assign it the <UNK> vector and count as OOV
        embedding_matrix[word] = unk_vector  # Assign OOV words the <UNK> vector
        oov_count += 1  # Increment OOV counter

# Print results for Word2Vec
# print(f"Number of OOV words with Word2Vec: {oov_count}")
# print(f"Embedding for <PAD>: {embedding_matrix['<PAD>']}")
# print(f"Embedding for <UNK>: {embedding_matrix['<UNK>']}")


In [ ]:
# Load pretrained FastText model (wiki-news-300d-subword)
fasttext_model = api.load('fasttext-wiki-news-subwords-300')

# Set embedding size
embedding_size = 300

# Initialize the embedding matrix with zeros for padding and random values for unknown tokens
embedding_matrix = {}

# Create an <UNK> token embedding as a random vector
unk_vector = np.random.uniform(-0.25, 0.25, embedding_size)
embedding_matrix["<UNK>"] = unk_vector

# Create a <PAD> token embedding as a zero vector
pad_vector = np.zeros(embedding_size)
embedding_matrix["<PAD>"] = pad_vector

# Initialize OOV counter for FastText
oov_count_fasttext = 0

# Iterate over the vocabulary
for word in vocab:
    if word == "<PAD>" or word == "<UNK>":
        continue  
    
    try:
        # Try to get the word vector using FastText's subword handling
        embedding_matrix[word] = fasttext_model.get_vector(word)
    except KeyError:
        # If the word can't be processed even by FastText, assign it the <UNK> vector
        embedding_matrix[word] = unk_vector
        oov_count_fasttext += 1  # Increment OOV count

# Print results for FastText
print(f"Number of OOV words with FastText: {oov_count_fasttext}")
print(f"Embedding for <PAD>: {embedding_matrix['<PAD>']}")
print(f"Embedding for <UNK>: {embedding_matrix['<UNK>']}")

In [9]:
#store the embeddings so that they can be used later
np.save("embedding_matrix.npy", embedding_matrix)


## Part 3.3.Keeping the above two adjustments, replace your simple RNN model in Part 2 wioth a biLSTM model and biGRU model.

# biLSTM Model

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score

preparing the train dataset. text -> word index

In [7]:
embedding_matrix_33=np.load("embedding_matrix.npy",allow_pickle='TRUE').item()
# display(embedding_matrix_33)

In [8]:
class SentimentDataset_33(Dataset):
    def __init__(self, texts : list[str], labels : list[int], vocab : set, embedding_matrix : dict, max_len=30):
        self.texts : list[str] = texts
        self.labels : list[int] = labels
        self.vocab : dict = self.build_vocab_dict(vocab)  # function to build vocabulary
        self.embedding_matrix : dict = embedding_matrix
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized_text = word_tokenize(text.lower())
        vectorized_text = self.vectorize(tokenized_text)
        return torch.tensor(vectorized_text, dtype=torch.long), torch.tensor(label, dtype=torch.float)

    def vectorize(self, tokens):
        # Convert tokens to their corresponding index in the vocabulary
        vectorized = [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]

        # Pad or truncate to max_len
        if len(vectorized) < self.max_len:
            vectorized += [self.vocab['<PAD>']] * (self.max_len - len(vectorized))
        else:
            vectorized = vectorized[:self.max_len]
        return vectorized

    def build_vocab_dict(self, vocab : set):
        if "<PAD>" in vocab : vocab.remove("<PAD>")
        if "<UNK>" in vocab : vocab.remove("<UNK>")
        vocab_dict = {word: idx for idx, word in enumerate(vocab)}
        vocab_dict['<PAD>'] = len(vocab_dict) # Add padding token
        vocab_dict['<UNK>'] = len(vocab_dict) # Add unknown token
        return vocab_dict


make the bilstm

In [10]:
class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, dropout_rate):
        super(SentimentBiLSTM, self).__init__()
        # Load pre-trained embeddings
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.size(1), hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        # print("LSTM out: ", lstm_out.shape)
        # Get the final forward and backward hidden states
        out = torch.cat((lstm_out[:, -1, :self.lstm.hidden_size], lstm_out[:, 0, self.lstm.hidden_size:]), dim=1)
        # print(out)
        out = self.dropout(out)
        # return self.sigmoid(self.fc(out))
        return self.fc(out)

### Training, evaluation functions

training function

In [9]:
def train_33(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in iterator:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU
        optimizer.zero_grad()
        output = model(X_batch).squeeze(1)
        loss = criterion(output, y_batch.float())
        loss.backward()
        # for param in model.parameters():
        #     if param.grad is not None:  # Ensure the gradient is not None
        #         print(f"Gradient norm for {param.shape}: {param.grad.data.norm()}")
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)


evaluation function

In [10]:
def evaluate_33(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in iterator:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU
            output = model(X_batch).squeeze(1)
            
            probs = model.sigmoid(output)
            prediction = (probs >= 0.5).float()
            
            loss = criterion(output, y_batch.float())

            epoch_loss += loss.item()
            all_preds.extend(prediction.tolist())
            all_labels.extend(y_batch.tolist())
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy, epoch_loss / len(iterator)


train and validate function

In [11]:
def train_and_validate_33(num_epochs, model, train_iterator, valid_iterator, optimizer, criterion, scheduler):
    for epoch in range(num_epochs):
        train_loss = train_33(model, train_iterator, optimizer, criterion)
        accuracy , valid_loss = evaluate_33(model, valid_iterator, criterion)
        scheduler.step(valid_loss)
        if (epoch==0):
                best_val_loss = valid_loss
                epochs_without_improvement = 0
        print(f'Epoch {epoch + 1}: Train Loss = {train_loss:.3f}, Accuracy = {accuracy:.3f}, Val Loss = {valid_loss:.3f} Learning Rate: {scheduler.optimizer.param_groups[0]["lr"]:.6f}')

        if valid_loss < best_val_loss:
            best_val_loss = valid_loss
            epochs_without_improvement = 0  # Reset counter
        else:
            epochs_without_improvement += 1
            
        # Check for convergence
        if epochs_without_improvement >= 8:  # Convergence condition (no improvement for 4 epochs)
            print("Convergence reached, stopping training.")
            break

create model and run the train loop

In [12]:
# Prepare your datasets
train_texts_33 : list[str] = train_dataset['text']  # List of training texts
train_labels_33 : list[int] = train_dataset['label']  # Corresponding labels for training texts
valid_texts_33 : list[str]= validation_dataset['text']  # List of validation texts
valid_labels_33 : list[int] = validation_dataset['label']  # Corresponding labels for validation texts
test_texts_33 : list[str] = test_dataset['text']  # List of test texts
test_labels_33 : list[int] = test_dataset['label']  # Corresponding labels for test texts
vocab_33 : set = vocab  # Your vocabulary list
embedding_matrix_33 : dict[ str , np.ndarray]= np.load("embedding_matrix.npy",allow_pickle='TRUE').item()
embedding_matrix_values = np.array(list(embedding_matrix_33.values()), dtype=np.float32)
embedding_matrix_tensor = torch.tensor(embedding_matrix_values, dtype=torch.float32)

# Create dataset instances
train_dataset_33 : SentimentDataset_33 = SentimentDataset_33(train_texts_33, train_labels_33, vocab_33, embedding_matrix_33)
valid_dataset_33 : SentimentDataset_33 = SentimentDataset_33(valid_texts_33, valid_labels_33, vocab_33, embedding_matrix_33)
test_dataset_33 : SentimentDataset_33 = SentimentDataset_33(test_texts_33, test_labels_33, vocab_33, embedding_matrix_33)

# Create data loaders
train_iterator_33 = DataLoader(train_dataset_33, batch_size=32, shuffle=True)
valid_iterator_33 = DataLoader(valid_dataset_33, batch_size=32, shuffle=False)
test_iterator_33 = DataLoader(test_dataset_33, batch_size=32, shuffle=False)


In [13]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [16]:
hidden_dim = 128  # Adjust as needed
output_dim = 1  # Binary sentiment classification
model_33 = SentimentBiLSTM(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate=0.0).to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_33.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4)
# Now you can run your training loop
train_and_validate_33(25, model_33, train_iterator_33, valid_iterator_33, optimizer, criterion, scheduler)

test

## Hyper-parameter Training

In [ ]:
import itertools

# Define the hyper-parameter grid
hidden_dims = [64, 128 ,256]
learning_rates = [0.001, 0.005]
dropout_rates = [0.3, 0.5]
batch_sizes = [32, 64]
output_dim = 1 

# Iterate over all combinations of hyper-parameters
for hidden_dim, lr, dropout_rate, bs in itertools.product(hidden_dims, learning_rates, dropout_rates, batch_sizes):
    print(f'Training with hidden_dim={hidden_dim}, lr={lr}, dropout_rate={dropout_rate}, batch_size={bs}')
    
    model_33 = SentimentBiLSTM(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model_33.parameters(), lr=lr)

    train_iterator_33 = DataLoader(train_dataset_33, bs)
    valid_iterator_33 = DataLoader(valid_dataset_33, bs)

    train_and_validate_33(25, model_33, train_iterator_33, valid_iterator_33, optimizer, criterion)


## Training with best parameters : 
hidden_dim=64, 

lr=0.001, 

dropout_rate=0.3, 

batch_size=64

In [33]:
hidden_dim = 64
lr=0.001
dropout_rate=0.3
bs=64
output_dim = 1 

model_33 = SentimentBiLSTM(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_33.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4)

train_iterator_33 = DataLoader(train_dataset_33, bs)
valid_iterator_33 = DataLoader(valid_dataset_33, bs)

train_and_validate_33(25, model_33, train_iterator_33, valid_iterator_33, optimizer, criterion, scheduler)


NameError: name 'SentimentBiLSTM' is not defined

## Testing

In [31]:
# Converting the Vocab set to dictionary 
def build_vocab_dict(vocab_set):
    # Create the vocabulary dictionary without <PAD> and <UNK>
    vocab_set.discard("<PAD>")
    vocab_set.discard("<UNK>")
    vocab_dict = {word: idx for idx, word in enumerate(vocab_set, start=2)}

    # Check for <PAD> and <UNK> existence and assign them fixed indices if they are present
    if "<PAD>" not in vocab_dict:
        vocab_dict["<PAD>"] = 0  # Index for padding token
    if "<UNK>" not in vocab_dict:
        vocab_dict["<UNK>"] = 1  # Index for unknown token
    
    #add the <PAD> and <UNK> back to the vocab
    vocab_set.add("<PAD>")
    vocab_set.add("<UNK>")
    return vocab_dict


In [32]:
# Step 8: Get a sample sentence from the test set and predict
import random
# Select a random index from the test dataset
random_index = random.randint(0, len(test_dataset) - 1)

# Get the corresponding sentence and its label from the test dataset
sample_sentence = test_dataset[random_index]['text']  # Assuming the dataset contains a 'text' field
true_label = test_dataset[random_index]['label']  # Assuming there's a label field
# Tokenize the sample sentence
sample_tokens = word_tokenize(sample_sentence.lower())

vocab_33 = build_vocab_dict(vocab)
# Convert tokens to indices
sample_indices = []
for token in sample_tokens:
    if token in vocab:
        sample_indices.append(vocab_33[token])
    else:
        sample_indices.append(vocab_33['<UNK>'])

sample_tensor = torch.tensor(sample_indices).unsqueeze(0)  # Add batch dimension
sample_tensor = sample_tensor.to(device)  # Move to GPU if available
# Make prediction using the model
model_33.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to compute gradients during inference
    output = model_33(sample_tensor)  # Pass the tensor to the model
    print(output)
    probs = model_33.sigmoid(output)
    predicted = (probs >= 0.5)
    print(predicted.item())

# Map predicted index to sentiment label
sentiment_labels = ['negative', 'positive']  # Adjust according to your label encoding
predicted_label = sentiment_labels[predicted]

# Print results
print(f"Sample Sentence: '{sample_sentence}'")
print(f"True Label: {sentiment_labels[true_label]}")
print(f"Predicted Label: {predicted_label}")

NameError: name 'model_33' is not defined

In [19]:
def test_33(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in iterator:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU
            output = model(X_batch).squeeze(1)
            
            probs = model.sigmoid(output)
            prediction = (probs >= 0.5).float()
            
            loss = criterion(output, y_batch.float())

            epoch_loss += loss.item()
            all_preds.extend(prediction.tolist())
            all_labels.extend(y_batch.tolist())
            
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy = {accuracy:.3f}")



In [ ]:
test_33(model_33, test_iterator_33, criterion)

Accuracy = 0.744


## BiLSTM with an Attention Layer

In [14]:
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.attn = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, hidden, encoder_outputs):
        """
        hidden: [batch_size, hidden_dim] (last hidden state of the decoder or a step in the sequence)
        encoder_outputs: [batch_size, seq_len, hidden_dim] (outputs from the encoder)
        """
        # Compute the attention weights using the query (hidden state) and keys (encoder outputs)
        attn_weights = torch.matmul(encoder_outputs, hidden.unsqueeze(2)).squeeze(2)
        
        # Apply softmax to get the attention weights
        attn_weights = F.softmax(attn_weights, dim=1)
        
        # Compute the weighted sum of the encoder outputs (values)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context, 



In [15]:
class BiLSTMWithAttention(nn.Module):
    
    
    def __init__(self, embedding_matrix, hidden_dim, output_dim, dropout_rate):
        super(BiLSTMWithAttention, self).__init__()
        # Load pre-trained embeddings
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.size(1), hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.attn = Attention(hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):
        # Embed the input
        embedded = self.embedding(x) #[B, S, E]
        
        # Get the outputs and hidden states from the LSTM
        lstm_out,_= self.lstm(embedded)  # lstm_out: [batch_size, seq_len, hidden_dim * 2]
        
        # Use the last hidden state of the LSTM as the query for the attention mechanism
        last_hidden = lstm_out[:, -1, :] #[B, 2 * H]
        
        # Apply attention to the LSTM outputs
        context, attn_weights = self.attn(last_hidden, lstm_out) #[B, 2 * H], [B, S]
        
        # Pass the context vector through a fully connected layer
        output = self.fc(context) #[B, output_dim]
        
        return output


In [24]:
hidden_dim = 128
lr=0.001
dropout_rate=0.3
bs=64
output_dim = 1 


model_attn= BiLSTMWithAttention(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate ).to(device)
criterion_attn = nn.BCEWithLogitsLoss()
optimizer_attn = torch.optim.Adam(model_attn.parameters(), lr=1e-3)
scheduler_attn = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_attn, mode='min', factor=0.1, patience=4)

In [25]:
train_and_validate_33(2, model_attn, train_iterator_33,valid_iterator_33, optimizer_attn, criterion_attn, scheduler_attn)

Epoch 1: Train Loss = 0.629, Accuracy = 0.738, Val Loss = 0.532 Learning Rate: 0.001000
Epoch 2: Train Loss = 0.330, Accuracy = 0.748, Val Loss = 0.558 Learning Rate: 0.001000


In [26]:
test_33(model_attn, test_iterator_33, criterion_attn)

Accuracy = 0.774


In [29]:
train_and_validate_33(2, model_attn, train_iterator_33,valid_iterator_33, optimizer_attn, criterion_attn, scheduler_attn)
test_33(model_attn, test_iterator_33, criterion_attn)

Epoch 1: Train Loss = 0.012, Accuracy = 0.743, Val Loss = 1.345 Learning Rate: 0.000100
Epoch 2: Train Loss = 0.009, Accuracy = 0.741, Val Loss = 1.395 Learning Rate: 0.000100
Accuracy = 0.776


## Multi-head Self Attention

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        assert hidden_dim % num_heads == 0, "Hidden dimension must be divisible by the number of heads"
        self.num_heads = num_heads
        self.dim_per_head = hidden_dim // num_heads
        
        # Define linear layers for query, key, and value
        self.query_layer = nn.Linear(hidden_dim, hidden_dim)
        self.key_layer = nn.Linear(hidden_dim, hidden_dim)
        self.value_layer = nn.Linear(hidden_dim, hidden_dim)
        
        # Linear layer to combine heads
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, encoder_outputs):
        batch_size = encoder_outputs.size(0)
        
        # Linear projections
        query = self.query_layer(encoder_outputs)  # [batch_size, seq_len, hidden_dim]
        key = self.key_layer(encoder_outputs)
        value = self.value_layer(encoder_outputs)
        
        # Split into multiple heads and reshape for attention calculation
        query = query.view(batch_size, -1, self.num_heads, self.dim_per_head).transpose(1, 2)
        key = key.view(batch_size, -1, self.num_heads, self.dim_per_head).transpose(1, 2)
        value = value.view(batch_size, -1, self.num_heads, self.dim_per_head).transpose(1, 2)
        
        # Scaled Dot-Product Attention
        attn_weights = torch.matmul(query, key.transpose(-2, -1)) / (self.dim_per_head ** 0.5)
        attn_weights = F.softmax(attn_weights, dim=-1)
        
        # Compute weighted sum of values
        context = torch.matmul(attn_weights, value)
        
        # Concatenate heads and pass through final linear layer
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.hidden_dim)
        return self.fc(context)

## BiLSTM with Multi-head Self Attention 

In [ ]:
class BiLSTMWithSelfAttention(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, dropout_rate, num_heads=4):
        super(BiLSTMWithSelfAttention, self).__init__()
        # Load pre-trained embeddings
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        
        # Define the bidirectional LSTM layer
        self.lstm = nn.LSTM(embedding_matrix.size(1), hidden_dim, num_layers=1, 
                            bidirectional=True, batch_first=True)
        
        # Multi-head self-attention layer
        self.self_attn = MultiHeadSelfAttention(hidden_dim * 2, num_heads=num_heads)
        
        # Fully connected layer to output the final prediction
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # Embed the input
        embedded = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        
        # Pass the embeddings through the LSTM
        lstm_out, _ = self.lstm(embedded)  # lstm_out: [batch_size, seq_len, hidden_dim * 2]
        
        # Apply multi-head self-attention on the LSTM outputs
        context = self.self_attn(lstm_out)  # [batch_size, seq_len, hidden_dim * 2]
        
        # Use mean pooling over the sequence length for final sentence representation
        pooled_context = torch.mean(context, dim=1)
        
        # Pass through fully connected layer
        output = self.fc(self.dropout(pooled_context))  # [batch_size, output_dim]
        
        return output

In [ ]:
bilstm_self_attn= BiLSTMWithAttention(embedding_matrix_tensor, hidden_dim, output_dim, dropout_rate ).to(device)
criterion_attn = nn.BCEWithLogitsLoss()
optimizer_attn = torch.optim.Adam(bilstm_self_attn.parameters(), lr=1e-3)
scheduler_attn = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_attn, mode='min', factor=0.1, patience=4)

In [ ]:
train_and_validate_33(20, bilstm_self_attn, train_iterator_33,valid_iterator_33, optimizer_attn, criterion_attn, scheduler_attn)

In [33]:
test_33(bilstm_self_attn, test_iterator_33, criterion_attn)

Accuracy = 0.781
